In [18]:
import time, math, pdb
import numpy as np
import pandas as pd
from collections import defaultdict
import sys
import string
from numba import jit, njit, types, jitclass, typed, typeof, deferred_type, float64, int32, uint8
# from numba.experimental import jitclass
import logging

numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.DEBUG)

def log(color=None, message=None, messageType="" , endChar="\n", alt = True, nl = 1):
    if color == None:
        for i in range(nl):
            print()
        return
    colors = {
    'HEADER': '\033[95m',
    'OKBLUE': '\u001b[38;5;6m',
    'OKGREEN': '\033[92m',
    'WARNING': '\033[93m',
    'FAIL': '\033[91m',
    'ENDC': '\033[0m',
    'BOLD': '\033[1m',
    'UNDERLINE': '\033[4m',
    'BLACK': '\u001b[38;5;0m'}
    if messageType != "":
        messageType = f"({messageType})  "
    if not alt:
        print(f"{colors[color]}{messageType}{message}{colors['ENDC']}", end = endChar, flush=True)
    else:
        print(f"{colors[color]}{messageType}{colors['ENDC']}{colors['BLACK']}{message}{colors['ENDC']}", end = endChar, flush=True)


def sigmoid(x):
    return 1 / (1 + math.exp(-1*x))

def dsigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))


def tanh(x):
    return np.tanh(x)


def dtanh():
    return 1 - (np.tanh(x)**2)


def relu(x):
    return max(0, x)


def drelu(x):
    if x > 0:
        return 1
    else:
        return 0

@jit()
def activations(name, derivative=False):
    a = pd.Series(index=['sigmoid', 'dsigmoid'], data = [sigmoid, dsigmoid])
    if derivative:
        name = "d"+name
    return a[name]

def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()


spec = [
    ('origin', types.unicode_type),
    ('weight', float64),
    ('buffer', float64),
    ('destination', types.unicode_type),
]
@jitclass(spec)
class Edge:
    def __init__(self, aorigin, adestination, aweight = 0.5, abuffer = 0):
        '''
        Constructor for Edge
        
        Params:
            origin (Node): origin node of edge (connected to tail of edge)
            destination (Node): destination node of edge (connected to head of edge)
            weight (float): magnitude of causality
            buffer (N/A): NOT IMPLEMENTED
        
        Return:
            N/A
        '''
        self.weight = aweight
        self.buffer = abuffer
        self.origin = str(aorigin)
        self.destination = str(adestination)
    
    def __str__(self):
        return f"Weight ({self.origin} -> {self.destination}): {self.weight}"
    
    def __repr__(self):
        return self.__str__()
        
edge_type = deferred_type()
edge_type.define(Edge.class_type.instance_type)
spec = [
    ('simulationSteps', float64[:]),
    ('indegree', edge_type[:]),
    ('outdegree', edge_type[:]),
    ('name', uint8[:]),
    ('activation', uint8[:]),

]
@jitclass(spec)  
class Node:
    def __init__(self, activation, indegree, outdegree, name=f"GC", simulationSteps=[]):
        '''
        Constructor for Node
        
        Params:
            activation (string) - activation type to be used
            indegree (list of Edge) - Edges pointing into the node
            outdegree (list of Edge) - Edges pointing out of the node
            name (string) - unique name/identifier for node
            simulationSteps (list of floats) - past simulation values, if any
        
        Return:
            N/A
        '''
        if len(simulationSteps) == 0:
            self.simulationSteps = np.array([0.0])
        else:
            self.simulationSteps = np.array(simulationSteps[:])               
        self.activation = activation
        self.indegree = np.array(indegree)
        self.outdegree = np.array(outdegree)
        self.name = name
    
    def getIndegreeNodes(self):
        '''
        Getter generator for nodes attached to indegree edges
        
        Params:
            N/A
        
        Yields:
            (node): nodes connected to indegree edges
        '''
        for edge in self.indegree.tolist():
            yield edge.origin
            
    def getOutdegreeNodes(self):
        '''
        Getter generator for nodes attached to outdegree edges
        
        Params:
            N/A
        
        Yields:
            (node): nodes connected to outdegree edges
        '''
        for edge in self.outdegree.tolist():
            yield edge.destination

    def updateValue(self, value):
        '''
        Update a value of a node
        
        Params:
            value (float): input to a node (pre-activation)
        
        Return:
            N/A
        '''
        np.append(self.simulationSteps, activations(self.activation)(value))
    
    def dryValue(self, value):
        '''
        Dry update a value of a node
        
        Params:
            value (float): input to a node (pre-activation)
        
        Return:
            (float): activated result
        '''
        return activations(self.activation)(value)
    
    def __str__(self):
        return f"{self.name}\n\tCurrent: {self.simulationSteps[-1]}\n\tIndegree: {self.indegree}\n\tOutdegree: {self.outdegree}\n\tHistory: {self.simulationSteps[:-1]}\n"
       
    def __repr__(self):
        return self.__str__()



node_type = deferred_type()
node_type.define(Node.class_type.instance_type)
spec = [
    ('nodes', node_type)
]
@jitclass(spec)
class Graph:
    def __init__(self, weightMatrix=[[0.1]], names = ['A'], activation='sigmoid'):
        '''
        Constructor for Graph (combination of Edges and Nodes)
        
        Params:
            nodes (list of Nodes): list of nodes if pre-created
            weightMatrix (Node): Pandas DataFrame consisting of a weight matrix of weight values where rows are origin node names and columns are destination node names

        Return:
            N/A
        '''
#         nodes = typed.Dict.empty(key_type=types.unicode_type, value_type=node_type)
        nodess = np.array([Node("activation",[],[], name= "index")])
        for i in range(len(names)):
            notindex = ""
            notindex = names[i]
            if i ==0:
                nodess[0] = Node(activation,[],[], name= notindex)
            else:
                np.append(nodess, np.array([Node(activation,[],[], name= notindex)]))
        for a, index in enumerate(weightMatrix):
            for b, column in enumerate(weightMatrix[a]):
                if not math.isnan(weightMatrix[a][b]):
                    weight = Edge(names[a], names[b], weightMatrix[a][b])
                    if len(nodess[a].outdegree) == 0:
                        nodess[a].outdegree = np.array([weight])
                    else:
                        nodess[a].outdegree = np.array(nodess[a].outdegree.tolist() + [weight])

                    if len(nodess[b].indegree) == 0:
                        nodess[b].indegree = np.array([weight])
                    else:
                        nodess[b].indegree = np.array(nodess[b].indegree.tolist() + [weight])
                    nodess[a].outdegree = np.append(nodess[a].outdegree, np.array([weight], dtype = edge_type))
                    nodess[b].indegree = np.append(nodess[b].indegree, np.array([weight]))
        self.nodes = np.array(nodess)
            
    def step(self):
        '''
        Runs an iteration of graph simulation
        
        Params:
            N/A
        
        Return:
            (Pandas Series): Series of Node values from after simulation step
        '''
        index = len(self.nodes[0].simulationSteps)-1
        res = pd.Series(index = [node.name for node in self.nodes], dtype="float64")
        for node in self.nodes.tolist():
            vals = np.array([self.getNode(item).simulationSteps[index] for item in node.getIndegreeNodes()])
            weights = np.array([self.getNode(item).weight for item in node.indegree.tolist()])
            node.updateValue(np.sum(np.multiply(vals, weights)))
            res[node.name] = node.simulationSteps[-1]
        return res
    
    def getNode(self, val):
        for a in self.nodes:
            if a.name == val:
                return a
        return None
    
    def dryStep(self, inputs):
        '''
        Dry runs an iteration of graph simulation
        
        Params:
            N/A
        
        Return:
            (Pandas Series): Series of Node values from after simulation step
        '''
        res = pd.Series(index = [node.name for node in self.nodes.tolist()], dtype="float64")
        for node in self.nodes.tolist():
            vals = np.array([inputs[item] for item in list(node.getIndegreeNodes())])
            weights = np.array([item.weight for item in node.indegree.tolist()])
            res[node.name] = node.dryValue(np.sum(np.multiply(vals, weights)))
        return res
    

    def runSteps(self, numSteps, probability=-1, numNodes=0):
        '''
        Runs multiple iterations of graph simulation with randomization
        
        Params:
            numSteps (integer): number of steps to simulate
            probability (float): float between 0 and 1 for chance of randomizing nodes at any given step
            numNodes (int): number of node values to randomize if randomization occurs
        
        Return:
            N/A
        '''
        for i in range(numSteps):
            self.step()
            if np.random.random() < probability and len(self.nodes[0].simulationSteps)>0:
                for _ in range(numNodes):
                    self.nodes[np.random.randint(0,len(self.nodes))].simulationSteps[-1] = np.random.random()

            
    def getWeightMatrix(self):
        '''
        Getter method for current Edge matrix
        
        Params:
            N/A
        
        Return:
            (Pandas DataFrame): Edge matrix of Graph
        '''
        weights = [weight for node in self.nodes.tolist() for weight in node.indegree.tolist()]
        df = pd.DataFrame( index = [node.name for node in self.nodes.tolist()], columns = [node.name for node in self.nodes.tolist()])

        for weight in weights:
            try:
                df.loc[weight.origin][weight.destination] = weight.weight
            except:
                print(weight)
        return df
    
    def getWeights(self):
        '''
        Getter method for Edges connecting Nodes
        
        Params:
            N/A
        
        Return:
            (List of Edges): Edges used in Graph
        '''
        return [weight for node in self.nodes.tolist() for weight in node.indegree.tolist()]
    
    def getNodes(self):
        '''
        Getter method for Nodes in Graph
        
        Params:
            N/A
        
        Return:
            (Pandas DataFrame): DataFrame consisting of Node names as column heads and each row is a simulation step
        '''
        df = pd.DataFrame(index = list(range(len(self.nodes[0].simulationSteps))), columns = [node.name for node in self.nodes.tolist()])
        for node in self.nodes:
            df[node.name] = node.simulationSteps
        return df
    
    def __str__(self):
        res = ""
        for node in self.nodes:
            res += str(node) + "\n\n"
        return res
    
    def __repr_s_(self):
        return self.__str__()

class Training:
    @njit()
    def runOSDR(graph, data, learningRate=0.2):
        '''
        OSDR = One Step Delta Rule (creds go to M. Gregor and P.P. Groumpos)
        A supervised algorithm that applies gradient descent to FCM (no backprop)
        
        Params:
            graph (Graph): Graph of Nodes and Edges
            data (Pandas DataFrame): DataFrame of training data with Node names as column heads and each row representing an additional sample
            learningParam (float): hyperparameter for training
        
        Return:
            N/A
        '''
        nodes = graph.nodes
        for progress, i in enumerate(range(1, data.shape[0],2)):
            weights = graph.getWeights()
            for a, weight in enumerate(weights):
                error = data.iloc[i] - graph.dryStep(data.iloc[i-1]) 
                node=""
                for nodep in nodes:
                    if nodep.name == weight.destination:
                        node = nodep
                        break
                vals = np.array([data.iloc[i-1][item] for item in node.getIndegreeNodes()])
                weightsarr = np.array([item.weight for item in node.indegree.tolist()])
                tot = np.sum(np.multiply(vals, weightsarr))
                du = activations(node.activation, True)(tot)
                weight.weight += learningRate * (error[weight.destination] * du) * (data.iloc[i-1][weight.origin])
#             print(progress)
  

In [19]:
SAMPLES = 300
SIZE = 3
LEARNING_COEFFICIENT = 0.2
MIN_WEIGHT = -2
MAX_WEIGHT = 2
np.random.seed(2)

log("OKBLUE", "Setup start", "info")
start = time.time()
def namer(i):
    res = [""]
    al = string.ascii_uppercase
    for a in range(i):
        res.append(al[a%len(al)]+str(math.floor(a/len(al))+1))
    return res[1:]
refArr = (np.random.randint(MIN_WEIGHT*10, (MAX_WEIGHT*10)+1, SIZE**2)/10).reshape(SIZE,SIZE)
refMatrix = pd.DataFrame(data = refArr,index = namer(SIZE), columns = namer(SIZE))
np.fill_diagonal(refMatrix.values, np.nan)
resArr = (np.zeros(SIZE**2)).reshape(SIZE,SIZE)
resMatrix = pd.DataFrame(data = resArr,index = namer(SIZE), columns = namer(SIZE))
np.fill_diagonal(resMatrix.values, np.nan)

# refArr = [[math.nan,0.5,0.1,],[0.7,math.nan,0.3 ],[0.9,0.2,math.nan]]
# resArr = [[math.nan,0.0,0.0 ],[0.0,math.nan,0.0 ],[0.0,0.0,math.nan]]
# refMatrix = pd.DataFrame(data = refArr,index = list("ABC"), columns = list("ABC"))
# resMatrix = pd.DataFrame(data = resArr,index = list("ABC"), columns = list("ABC"))
refModel = Graph(weightMatrix = refMatrix.to_numpy(), activation='sigmoid', names=list(refMatrix.index))
resModel = Graph(weightMatrix = resMatrix.to_numpy(), activation='sigmoid', names=list(refMatrix.index))
log("BLACK", "Reference Matrix")
log("BLACK", str(refModel.getWeightMatrix()))
log()
log("BLACK", "Untrained Matrix")
log("BLACK", str(resModel.getWeightMatrix()))
end = time.time()
log("OKBLUE", f"Setup completed in {round((end-start)*1000, 2)} ms", "info")

log(nl=2)

log("OKBLUE", "Training data creation start", "info")
start = time.time()
trainingData = pd.DataFrame(columns = namer(SIZE))
printProgressBar(0, SAMPLES, prefix = 'Progress:', suffix = 'Complete', length = 50)
for i in range(SAMPLES):
    initialVals = [np.random.uniform(0,1) for i in refModel.nodes.tolist()]
    before = pd.Series(data = initialVals, index = [node.name for node in refModel.nodes.tolist()], dtype="float64")
    after = refModel.dryStep(before)
    trainingData = trainingData.append([before,after], ignore_index=True)
    printProgressBar(i + 1, SAMPLES, prefix = 'Progress:', suffix = 'Complete', length = 50)
end = time.time()
log("OKBLUE", f"{int(len(trainingData.index)/2)} samples populated in {round((end-start)*1000, 2)} ms", "info")
log(nl=2)
print(trainingData)
log("OKBLUE", "Training start", "info")
start = time.time()
Training.runOSDR(resModel, trainingData, LEARNING_COEFFICIENT)
end = time.time()
log("OKBLUE", f"Training completed in {round((end-start)*1000, 2)} ms", "info")
log(nl=2)

log("OKGREEN", "", "results")
log("BLACK", "Reference Matrix")
log("BLACK", str(refModel.getWeightMatrix()))
log()
log("BLACK", "Trained Matrix")
log("BLACK", resModel.getWeightMatrix())



(info)  Setup start


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mInvalid use of Function(<built-in function array>) with argument(s) of type(s): (list(instance.jitclass.Edge#7fb43057ac10<origin:unicode_type,weight:float64,buffer:float64,destination:unicode_type>))
 * parameterized
[1mIn definition 0:[0m
[1m    NotImplementedError: instance.jitclass.Edge#7fb43057ac10<origin:unicode_type,weight:float64,buffer:float64,destination:unicode_type> cannot be represented as a Numpy dtype[0m
    raised from /Users/aku/opt/anaconda3/lib/python3.7/site-packages/numba/numpy_support.py:152
[1mIn definition 1:[0m
[1m    NotImplementedError: instance.jitclass.Edge#7fb43057ac10<origin:unicode_type,weight:float64,buffer:float64,destination:unicode_type> cannot be represented as a Numpy dtype[0m
    raised from /Users/aku/opt/anaconda3/lib/python3.7/site-packages/numba/numpy_support.py:152
[1mThis error

In [ ]:
resModel.nodes[0].outdegree

In [17]:
[1,2]+[3]

[1, 2, 3]